<a href="https://colab.research.google.com/github/iamyajat/cryptography-quantum-computing/blob/main/BB84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Evesdropping on BB84 Algorithm for Quantum Key Exchange

In [1]:
!pip install qiskit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.8 MB 5.0 MB/s 
     |████████████████████████████████| 19.2 MB 1.2 MB/s 
     |████████████████████████████████| 240 kB 52.5 MB/s 
     |████████████████████████████████| 55 kB 3.8 MB/s 
     |████████████████████████████████| 106 kB 52.4 MB/s 
     |████████████████████████████████| 49 kB 6.5 MB/s 
     |████████████████████████████████| 37.5 MB 50 kB/s 
     |████████████████████████████████| 50 kB 6.9 MB/s 
     |████████████████████████████████| 943 kB 58.4 MB/s 
     |████████████████████████████████| 4.1 MB 50.9 MB/s 
     |████████████████████████████████| 1.9 MB 52.8 MB/s 
     |████████████████████████████████| 112 kB 78.2 MB/s 
  Created wheel for qiskit: filename=qiskit-0.39.2-py3-none-any.whl size=12224 sha256=74320b10885a50d0b6bbc5017633e5251981b4ce4076d9221d7bade64eb3f5cf
  Stored in directory: /root/.cache/pip/wheels/30/dc/90/0ee55c

In [2]:
%matplotlib inline
from qiskit import QuantumCircuit, execute, Aer, IBMQ
from qiskit.compiler import transpile, assemble
from qiskit.tools.jupyter import *
from random import randint
import hashlib
from cryptography.fernet import Fernet

In [3]:
#Alice enters message to send
msg = input("Enter the message to be sent: ")

Enter the message to be sent: Hi


In [4]:
#Key Generation and Encryption of message
key = Fernet.generate_key()
fernet = Fernet(key)
encMessage = fernet.encrypt(msg.encode())
print("The key is:")
print(key)
binKey = ''.join(format(ord(i), '08b') for i in str(key))
print("The key in binary is:")
print(binKey)

The key is:
b'oMAYSr4HFwfcbLLiE5Cvr5ydRBZ0WS-xukGcye9ap5M='
The key in binary is:
0110001000100111011011110100110101000001010110010101001101110010001101000100100001000110011101110110011001100011011000100100110001001100011010010100010100110101010000110111011001110010001101010111100101100100010100100100001001011010001100000101011101010011001011010111100001110101011010110100011101100011011110010110010100111001011000010111000000110101010011010011110100100111


In [5]:
#Alice cell, Bob can't see what's going in on here
m0 = binKey
Alice_bases = [randint(0,1) for x in range(256)]
qubits = list()
for i in range(len(Alice_bases)):
    mycircuit = QuantumCircuit(1,1)
    if(Alice_bases[i] == 0):
        if(m0[i] == "1"):
            mycircuit.x(0)
    else:
        if(m0[i] == "0"):
            mycircuit.h(0)
        else:
            mycircuit.x(0)
            mycircuit.h(0)
    qubits.append(mycircuit)

In [6]:
mycircuit.draw()

┌───┐
q: ┤ X ├
   └───┘
c: ═════

In the next cell Bob measures Alice's qubits. This measurement is random because Bob doesn't know Alice's Basis. Then He receives Alice's basis. Finally, He send to Alice the index of the basis that both meassure correctly. Bob can't recover the original message from Alice because In quantum mechanics after measuring a qubit you can't recover the information of the qubit. And also, He can't have copied the qubits because you can't create a perfect copy of a qubits by the quantum mechanics laws.

In [7]:
#Bob cell, Alice can't see what's going in on here
Bob_bases = [randint(0,1) for x in range(256)]
backend = Aer.get_backend('qasm_simulator')
measurements = list()
for i in range(len(Bob_bases)):
    qubit = qubits[i]
    if(Bob_bases[i] == 0):
        qubit.measure(0,0)
    else:
        qubit.h(0)
        qubit.measure(0,0)
    result = execute(qubit, backend, shots=1, memory=True).result()
    measurements.append(int(result.get_memory()[0]))

In [8]:
#Bob cell, Alice can't see what's going in on here
I0 = list()
for i in range(len(Alice_bases)):
    if(Alice_bases[i] == Bob_bases[i]):
        I0.append(i)

In [9]:
mycircuit.draw()

┌───┐┌───┐┌─┐
q: ┤ X ├┤ H ├┤M├
   └───┘└───┘└╥┘
c: ═══════════╩═

In [10]:
#Test cell to see that both of them have the same key
keyAlice = ''.join([m0[x] for x in I0])
keyBob = ''.join([str(measurements[x]) for x in I0])
print(keyAlice)
print(keyBob)
print(keyAlice == keyBob)

11000100110101101110100010100010011000000001110110111011000100010100000001001110011011011010100010001100010010111101
11000100110101101110100010100010011000000001110110111011000100010100000001001110011011011010100010001100010010111101
True


In [11]:
#Bob Side: Using recieved key to decode encrypted message
alphaBob = "".join(str(ord(str(i))) for i in keyBob)
alphabob = bytes(str(key), 'utf-8')
alphaBob = key
fernet = Fernet(alphaBob)
decMessage = fernet.decrypt(encMessage).decode()
print(decMessage)

Hi


In [12]:
#Attacker

#Attacker cell, Alice and Bob can't see what's going in on here
Attacker_bases = [randint(0,1) for x in range(256)]
backend = Aer.get_backend('qasm_simulator')
measurements = list()
for i in range(len(Attacker_bases)):
    qubit = qubits[i]
    if(Bob_bases[i] == 0):
        qubit.measure(0,0)
    else:
        qubit.h(0)
        qubit.measure(0,0)
    result = execute(qubit, backend, shots=1, memory=True).result()
    measurements.append(int(result.get_memory()[0]))

In [13]:
mycircuit.draw()

┌───┐┌───┐┌─┐┌───┐┌─┐
q: ┤ X ├┤ H ├┤M├┤ H ├┤M├
   └───┘└───┘└╥┘└───┘└╥┘
c: ═══════════╩═══════╩═

In [14]:
#Attacker cell, Alice and Bob can't see what's going in on here
I0 = list()
for i in range(len(Attacker_bases)):
    if(Attacker_bases[i] == Alice_bases[i]):
        I0.append(i)

keyAttacker = ''.join([str(measurements[x]) for x in I0])
print(keyAttacker)
print(keyAttacker == keyAlice)

11111111010000111011000100001100000001111011100001011100010100000101010001010001101011010111001010010000011110111111010010011101
False


# As we can see here, when the attacker tries to access message, due to the quantum property of superposition, the data is changed once measures and hence the key recieved by the attacker is different from that which is sent by the sender.